In [1]:
import nltk
import string
import os
import pandas as pd
import numpy as np
from ggplot import *
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import re
from collections import Counter
import digify
import regex

In [2]:
#All classes
class BaseReplacer:
    def __init__(self, pattern_replace_pair_list=[]):
        self.pattern_replace_pair_list = pattern_replace_pair_list
    def transform(self, text):
        for pattern, replace in self.pattern_replace_pair_list:
            try:
                text = regex.sub(pattern, replace, text)
            except:
                pass
        return regex.sub(r"\s+", " ", text).strip()


    
class UnitConverter(BaseReplacer):
    """
    shadeMature height: 36 in. - 48 in.Mature width
    PUT one UnitConverter before LowerUpperCaseSplitter
    """
    def __init__(self):
        self.pattern_replace_pair_list = [
            (r"([0-9]+)( *)(inches|inch|in|in.|')\.?", r"\1 in. "),
            (r"([0-9]+)( *)(pounds|pound|lbs|lb|lb.)\.?", r"\1 lb. "),
            (r"([0-9]+)( *)(foot|feet|ft|ft.|'')\.?", r"\1 ft. "),
            (r"([0-9]+)( *)(square|sq|sq.) ?\.?(inches|inch|in|in.|')\.?", r"\1 sq.in. "),
            (r"([0-9]+)( *)(square|sq|sq.) ?\.?(feet|foot|ft|ft.|'')\.?", r"\1 sq.ft. "),
            (r"([0-9]+)( *)(cubic|cu|cu.) ?\.?(inches|inch|in|in.|')\.?", r"\1 cu.in. "),
            (r"([0-9]+)( *)(cubic|cu|cu.) ?\.?(feet|foot|ft|ft.|'')\.?", r"\1 cu.ft. "),
            (r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1 gal. "),
            (r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1 oz. "),
            (r"([0-9]+)( *)(centimeters|cm)\.?", r"\1 cm. "),
            (r"([0-9]+)( *)(milimeters|mm)\.?", r"\1 mm. "),
            (r"([0-9]+)( *)(minutes|minute)\.?", r"\1 min. "),
            (r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. "),
            (r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. "),
            (r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. "),
            (r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. "),
            (r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. "),
            (r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr "),
            (r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", r"\1 gal. per min. "),
            (r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr "),
        ]



In [3]:
#Extra Functions
def remove_commas_and_quotes(mystring):
    return re.sub(r'(?:(\d+?),)',r'\1',mystring)

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def lastlower(inputString):
    pos=-1
    for char in range(0,len(inputString)-1):
        if inputString[char].islower() and inputString[char+1].isdigit():
            pos=char
    return pos

def remove_puncs(word):
    if hasNumbers(word):
        p = re.compile("[^a-zA-Z0-9\.\-\/\,]")
        return p.sub(" ", word)
    else:
        p = re.compile("[^a-zA-Z0-9]")
        return p.sub(" ", word)
    
def splitdigits(word):
    if lastlower(word)==-1:
        return word
    else:
        return word[0:lastlower(word)+1]+" "+word[lastlower(word)+1:len(word)]
        

def di_digify(words):
    try:
        digify.spelled_num_to_digits(words)
        return digify.spelled_num_to_digits(words)
    except:
        return words

def digify_check(word):
     try:
        digify.spelled_num_to_digits(word)
        return 1
     except:
        return 0

def digitconverter(inputstr):
    words = inputstr.split()
    digit_words=[]
    words_mod=[]           
    for i in range(0,len(words)):
        digit_words.append(digify_check(words[i]))

    i=0
    while i<len(words)-1:
        if digit_words[i]+digit_words[i+1]==2:
            words_mod.append(digify.spelled_num_to_digits(words[i]+" "+words[i+1]))
            i=i+2
        else:
            words_mod.append(words[i])
            i=i+1
    if digify_check(words[len(words)-1])==0:
        words_mod.append(words[len(words)-1])
    digit_words=[]

    for i in range(0,len(words_mod)):
        digit_words.append(str(di_digify(str(words_mod[i]))))
    return(' '.join(digit_words))




In [4]:
#Load files
product=pd.read_csv("train.csv")
description=pd.read_csv("product_descriptions.csv")
#product_desc=pd.merge(product, description)
product_desc=product.copy()
attributes=pd.read_csv("attributes.csv")
attributes=attributes.dropna()
#product_desc.head()

In [5]:
#Extract brand and color
attributes['name'] = attributes['name'].str.lower()
brand = attributes[attributes['name'] =="mfg brand name"]
brand=brand.rename(columns={"value": "product_brand"})
brand.drop('name', axis=1, inplace=True)
color_columns = ["product_color", "color family", "color/finish", "color/finish family"]
color = attributes[attributes['name'].isin(color_columns)][["product_uid", "value"]].rename(columns={"value": "product_color"})
color = color.dropna()
agg_color = lambda x: " ".join(list(set(x["product_color"])))
color = color.groupby("product_uid").apply(agg_color)
color = color.reset_index(name="product_color")
color["product_color"] = color["product_color"].values.astype(str)
description = pd.merge(description, brand, on="product_uid", how="left")
description = pd.merge(description, color, on="product_uid", how="left")
description["product_brand"] = description["product_brand"].astype(str)
description["product_color"] = description["product_color"].astype(str)

In [6]:
attributes["product_uid"] = attributes["product_uid"].astype(str)
attributes["product_attribute_concat"]=attributes["name"]+" : "+attributes["value"]
attr_concat=attributes.groupby('product_uid')['product_attribute_concat'].apply(' | '.join).reset_index()
attr_concat["product_uid"] = attr_concat["product_uid"].astype(float)
attr_concat["product_uid"] = attr_concat["product_uid"].astype(int)
description = pd.merge(description, attr_concat, on="product_uid", how="left")
description["product_attribute_concat"] = description["product_attribute_concat"].astype(str)
#description.head()

In [7]:
#Removing comma from digits
product_desc['product_title'] = product_desc['product_title'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))
product_desc['search_term'] = product_desc['search_term'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))
description['product_brand'] = description['product_brand'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))
description['product_color'] = description['product_color'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))

description['product_description'] = description['product_description'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))

description['product_attribute_concat'] = description['product_attribute_concat'].apply(lambda x:  " ".join\
                                                                ([remove_commas_and_quotes(word) for word in x.split(" ")]))


In [8]:
#Removing punctuations
product_desc['product_title'] = product_desc['product_title'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))
product_desc['search_term'] = product_desc['search_term'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))
description['product_brand'] = description['product_brand'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))
description['product_color'] = description['product_color'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))
description['product_description'] = description['product_description'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))
description['product_attribute_concat'] = description['product_attribute_concat'].apply\
(lambda x:  " ".join([remove_puncs(word) for word in x.split(" ")]))


In [9]:
#Seperate using caps

#splitting by case
product_desc['product_title']= product_desc['product_title'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

product_desc['search_term']= product_desc['search_term'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

description['product_brand']= description['product_brand'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

description['product_color']= description['product_color'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

description['product_description']= description['product_description'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

description['product_attribute_concat']= description['product_attribute_concat'].apply\
(lambda x:  ' '.join([str(elem) for elem in re.split(r'([A-Z][a-zA-Z]+)',x)]))

#splitting digits and alphabets


product_desc['product_title']= product_desc['product_title'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))

product_desc['search_term']= product_desc['search_term'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))

description['product_brand']= description['product_brand'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))

description['product_color']= description['product_color'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))

description['product_description']= description['product_description'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))

description['product_attribute_concat']= description['product_attribute_concat'].apply\
(lambda x:  " ".join([splitdigits(word) for word in x.split(" ")]))



In [10]:
#convert to lower
product_desc['product_title']=product_desc['product_title'].str.lower()
product_desc['search_term']=product_desc['search_term'].str.lower()
description['product_brand']=description['product_brand'].str.lower()
description['product_color']=description['product_color'].str.lower()
description['product_description']=description['product_description'].str.lower()
description['product_attribute_concat']=description['product_attribute_concat'].str.lower()


In [11]:
#Spell checker functions
spell_df = description
spell_df['constant']='group'
combine = spell_df.groupby('constant')['product_description'].apply(' '.join).reset_index()
concat=re.findall(r'\w+',combine['product_description'][0])


def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(concat)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [ ]:
#Spell checker
product_desc['product_title'] = product_desc['product_title'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
product_desc['search_term'] = product_desc['search_term'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
description['product_brand'] = description['product_brand'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
description['product_color'] = description['product_color'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
description['product_description'] = description['product_description'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
description['product_attribute_concat'] = description['product_attribute_concat'].apply\
(lambda x:  " ".join([correction(word) for word in x.split(" ")]))
description.drop('constant', axis=1, inplace=True)

In [ ]:
#Removing stop words
stop = stopwords.words("english")
product_desc['product_title'] = product_desc['product_title'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))
product_desc['search_term'] = product_desc['search_term'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))
description['product_brand'] = description['product_brand'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))
description['product_color'] = description['product_color'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))
description['product_description'] = description['product_description'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))
description['product_attribute_concat'] = description['product_attribute_concat'].apply\
(lambda x:  " ".join([word for word in x.split(" ") if word not in stop]))


In [ ]:
#Unit converter
product_desc['product_title'] = product_desc['product_title'].apply(lambda x: UnitConverter().transform(x))
product_desc['search_term'] = product_desc['search_term'].apply(lambda x: UnitConverter().transform(x))
description['product_brand'] = description['product_brand'].apply(lambda x: UnitConverter().transform(x))
description['product_color'] = description['product_color'].apply(lambda x: UnitConverter().transform(x))
description['product_description'] = description['product_description'].apply(lambda x: UnitConverter().transform(x))
description['product_attribute_concat'] = description['product_attribute_concat'].apply(lambda x: UnitConverter().transform(x))


In [ ]:
#Lemmatization and stemming
product_desc['product_title'] = product_desc['product_title'].apply(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(str(word))\
                                                                                         for word in x.split(" ")]))
#product_desc['product_title'] = product_desc['product_title'].apply(lambda x:  " ".join([stemmer.stem(word)\
#                                                                                         for word in x.split(" ")]))
product_desc['search_term'] = product_desc['search_term'].apply(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(word)\
                                                                                     for word in x.split(" ")]))
#product_desc['search_term'] = product_desc['search_term'].apply(lambda x:  " ".join([stemmer.stem(word)\
#                                                                                     for word in x.split(" ")]))
description['product_brand'] = description['product_brand'].apply(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(word) \
                                                                                         for word in x.split(" ")]))
#description['product_brand'] = description['product_brand'].apply(lambda x:  " ".join([stemmer.stem(word)\
#                                                                                         for word in x.split(" ")]))
description['product_color'] = description['product_color'].apply(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(word) \
                                                                                         for word in x.split(" ")]))
#description['product_color'] = description['product_color'].apply(lambda x:  " ".join([stemmer.stem(word)\
#                                                                                         for word in x.split(" ")]))

description['product_description'] = description['product_description'].apply\
(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(word) for word in x.split(" ")]))
#description['product_description'] = description['product_description'].apply\
#(lambda x:  " ".join([stemmer.stem(word) for word in x.split(" ")]))

description['product_attribute_concat'] = description['product_attribute_concat'].apply\
(lambda x:  " ".join([wordnet_lemmatizer.lemmatize(word) for word in x.split(" ")]))
#description['product_attribute_concat'] = description['product_attribute_concat'].apply\
#(lambda x:  " ".join([stemmer.stem(word) for word in x.split(" ")]))




In [ ]:
product_desc['product_title'] = product_desc['product_title'].str.replace(' x ','x')
product_desc['search_term']=product_desc['search_term'].str.replace(' x ','x')
description['product_brand']=description['product_brand'].str.replace(' x ','x')
description['product_color']=description['product_color'].str.replace(' x ','x')
description['product_description']=description['product_description'].str.replace(' x ','x')
description['product_attribute_concat']=description['product_attribute_concat'].replace(' x ','x')


In [ ]:
product_desc.to_csv("product_train_lemma.csv",sep=",",index=False)
description.to_csv("description_cleaned_lemma.csv",sep=",",index=False)